# LINEARISATION

In [1]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
import agrege_troncon as at
from Base_BdTopo.Import_outils import *
from Base_BdTopo.Rond_points import *
from Base_BdTopo.Regroupement_correspondance import *
from Outils import creer_graph
import Connexion_Transfert as ct
import matplotlib, os, fiona
import geopandas as gp
import pandas as pd
import numpy as np
from datetime import datetime
from collections import Counter
from sqlalchemy import Table, Column, Integer, String, Float,MetaData
from geoalchemy2 import Geometry
from shapely.wkt import dumps
pd.set_option('display.max_columns', 50)

## CD17
Réutiliser les comptages importés grace au Notebooj Import_trafics (chap cd17) et les données de troncons elemnetaires issues du notebook agrege_troncon pour déterminer les nouveaux id_comptag 

In [2]:
#import des données de troncon elem (à modifier par imporft depuis Bdd quand le transfert vers Bdd en sortie d'agreg_troncon sera calé) et 
#si besoin jointure avec les données de trafic
gdf_traf=gp.read_file(r'C:\Users\martin.schoreisz\Documents\AffairesEnCours\OTV\17\temp_linearisation\troncelem2019_Bdt17_ed19_l.shp')
gdf_lin=gp.read_file(r'C:\Users\martin.schoreisz\Documents\AffairesEnCours\OTV\17\temp_linearisation\traf2019_bdt17_ed19_l.shp')
gdf_traf=gdf_lin.merge(gdf_traf[['id_ign', 'idtronc']], on='id_ign')

In [150]:
#import des points de comptages depuis la Bdd
with ct.ConnexionBdd('local_otv_boulot') as c :
    rqt="select * from comptage.na_2010_2019_p where dep='17' and geom is not null and type_poste='ponctuel'"
    gdf_pt_cpt = gp.read_postgis(rqt, c.connexionPsy)
#mise en forme attributs
def convert_tmja(df) : 
    for annee in [a for a in range(2019, 2009,-1)]+['autr'] :
        attr='tmja_'+str(annee)
        if not pd.isnull(df[attr]) : 
            return (df[attr], annee)
df_adj=gdf_pt_cpt.apply(lambda x : convert_tmja(x), axis=1,result_type='expand')
df_adj.columns=['tmja_recent','annee_tmja_recent']
gdf_pt_cpt=gdf_pt_cpt.merge(df_adj, left_index=True, right_index=True).rename(columns={'id_voie_d':'codevoie_d'})#[['id_comptag','geom','type_poste','tmja_recent','annee_tmja_recent' ]]

In [151]:
#filtre des points ponctuels en été
pt_pctuels_ete=gdf_pt_cpt.loc[gdf_pt_cpt['type_poste']=='ponctuel'].copy()
def verif_ete(tmja_2015, obs_2015, tmja_2016, obs_2016,tmja_2017, obs_2017,tmja_2018, obs_2018) :
    if tmja_2018 > 0 and not pd.isnull(obs_2018) :
        return any([a in ['July','August']  for a in [pd.to_datetime(obs_2018.split(',')[1].split('-')[i], format='%d/%m/%Y').month_name()for i in [0,1]]])
    elif tmja_2017 > 0 and not pd.isnull(obs_2017) :
        return any([a in ['July','August']  for a in [pd.to_datetime(obs_2017.split(',')[1].split('-')[i], format='%d/%m/%Y').month_name()for i in [0,1]]])
    elif tmja_2016 > 0 and not pd.isnull(obs_2016) :
        return any([a in ['July','August']  for a in [pd.to_datetime(obs_2016.split(',')[1].split('-')[i], format='%d/%m/%Y').month_name()for i in [0,1]]])
    elif tmja_2015 > 0 and not pd.isnull(obs_2015) :
        return any([a in ['July','August']  for a in [pd.to_datetime(obs_2015.split(',')[1].split('-')[i], format='%d/%m/%Y').month_name()for i in [0,1]]])
    else : return False
pt_pctuels_ete['ete']=pt_pctuels_ete.apply(lambda x : verif_ete(x['tmja_2015'],x['obs_2015'], x['tmja_2016'], x['obs_2016'],
                                                               x['tmja_2017'],x['obs_2017'], x['tmja_2018'], x['obs_2018']), axis=1)
pt_pctuels_ete=pt_pctuels_ete.loc[~pt_pctuels_ete['ete']].copy()
gdf_pt_cpt=gdf_pt_cpt.loc[gdf_pt_cpt.index.isin(pt_pctuels_ete.index.tolist()+gdf_pt_cpt.loc[gdf_pt_cpt['type_poste']!='ponctuel'].index.tolist())].copy()

In [152]:
#croisement point et lignes (sur labase d'un buffer de 0.5m)
gdf_pt_cpt.geometry=gdf_pt_cpt.buffer(0.5)
gdf_traf_pt=gp.sjoin(gdf_traf,gdf_pt_cpt,how="left", op='intersects' )[['id','id_ign','tmja', 'importance','numero','id_voie_d','sens','long_km','nature',
            'id_comptag_left', 'idtronc', 'id_comptag_right','type_poste','tmja_recent','annee_tmja_recent','source','target', 'geometry']].rename(
columns={'id_comptag_left':'id_cpt_lin','id_comptag_right': 'id_cpt_pt','tmja':'tmja_lin','id_voie_d':'codevoie_d',
           'idtronc':'id_tronc_e'})

In [153]:
#recherch des tronc_elem avec plusieurs points
gdf_traf_pt_notna=gdf_traf_pt.loc[~gdf_traf_pt['id_cpt_pt'].isna()].copy()
tronc_elem_pt_cpt=gdf_traf_pt_notna.sort_index().groupby('id_tronc_e').agg({'id_cpt_pt' : lambda x : tuple(x),
                                       'type_poste' : lambda x : tuple(x),
                                        'tmja_recent' : lambda x : tuple(x),
                                        'annee_tmja_recent' : lambda x : tuple(x)})
trc_elem_multipt=tronc_elem_pt_cpt.loc[tronc_elem_pt_cpt.apply(lambda x : len(x['id_cpt_pt'])>1,axis=1)].copy()

In [154]:
#mise en forme des trocnon supportant plusieurs point : 
    #gestion des trocnon avec plusieurs points differntes (on garde le ponctuele de trafic max de l'annee la plus recente si que des ponctuels, sinon le tournant ou permanent)
    #verif de la validite des points en comprant les valeurs de trafic entre elle: si un des trafics est sup 1000 et que la variation est sup 30 %, on n'affecte pas
def cpt_final (type_poste,id_comptag_right,tmja_recent,annee_tmja_recent ): 
    tmja_recent=np.array(tmja_recent) 
    id_comptag_right=np.array(id_comptag_right)
    mask_tmja = (tmja_recent==max(tmja_recent))
    if all(np.unique(type_poste)=='ponctuel') :
        mask_annee=np.array(annee_tmja_recent)==max(annee_tmja_recent)
        if all(mask_annee):#il n'y a qu'une seule annee
            if all(mask_tmja) : 
                return id_comptag_right[0].tolist()
            else : 
                return id_comptag_right[mask_tmja][0].tolist()
        else : #il y a pluseurs annee dc on ne garde que le max de la bonne annee
            return id_comptag_right[mask_tmja].tolist()
    else :
        mask_type_poste=np.isin(np.array(type_poste),['permanent', 'tournant'])#on trouve le(s) poste(s) non ponctuel
        return id_comptag_right[mask_type_poste][0].tolist()
#verif de la validite des points en comprant les valeurs de trafic entre elle: si un des trafics est sup 1000 et que la variation est sup 30 %, on n'affecte pas
trc_elem_multipt['id_cpt_final']=trc_elem_multipt.apply(lambda x: cpt_final(
    x['type_poste'], x['id_cpt_pt'], x['tmja_recent'], x['annee_tmja_recent']), axis=1)
trc_elem_multipt['valide']=trc_elem_multipt.apply(lambda x : (False if abs(100-(min(x['tmja_recent'])/max(x['tmja_recent'])*100))>30 and
                                                                    max(x['tmja_recent'])>1000 and all(np.unique(x['type_poste'])=='ponctuel') else True), axis=1)
trc_elem_multipt=trc_elem_multipt.loc[trc_elem_multipt['valide']].copy()

In [156]:
#obtention de la df des id_comptage_pt par tronc_elem
tronc_elem_pt_cpt.update(trc_elem_multipt.drop('id_cpt_pt',axis=1).rename(columns={'id_cpt_final':'id_cpt_pt'})[['id_cpt_pt']])
tronc_elem_pt_cpt['id_cpt_pt']=tronc_elem_pt_cpt.id_cpt_pt.apply(lambda x : x[0] if (isinstance(x, tuple) or 
                                                       isinstance(x, list) and x) else x) #uniformisation du type de données en str

In [157]:
#transfert de l'id_cpt_pt final dans la df source
gdf_traf_pt_final=gdf_traf_pt[['id_ign','importance','numero', 'id_cpt_lin','tmja_lin', 'id_tronc_e','geometry','source','target']].merge(tronc_elem_pt_cpt[['id_cpt_pt']], how='left',left_on='id_tronc_e', right_index=True)

In [158]:
#pour info, ligne ign non linéarisée qui le deviennent
gdf_creation_idcpt=gdf_traf_pt_final.loc[(gdf_traf_pt_final['id_cpt_lin'].isna()) & (~gdf_traf_pt_final['id_cpt_pt'].isna())].copy()
#pour info, ligne ign linéarisée qui le sont toujours, mais peut avec un nouvel id_ign
gdf_variation_idcpt=gdf_traf_pt_final.loc[(~gdf_traf_pt_final['id_cpt_lin'].isna()) & (~gdf_traf_pt_final['id_cpt_pt'].isna()) & (gdf_traf_pt_final['id_cpt_pt']!=gdf_traf_pt_final['id_cpt_lin'])].sort_values('id_cpt_lin')

# mettre à jour les lignes des troncons qui appartiennent à des id_comptag qui supporte un nouvel id_cpt
**IL FAUT METTRE EN FORME LES 2 BLOCS CI-DESSSOUS EN FONCTION DANS LES MODULES QUI VONT BIEN <br> <br> ATTENTION DES ERREURS DE LINEARISATION EN 2016 :** <br>
- propagation non arreté à la 1èer intersection mais à la deuxième (à l'Est de l'id_comptag) : TRONROUT0000000032986160 linearisee avec 17-D129-57+510
- rd point linéarisé mais aucun autre troncon avec lui : 17-D202-9+950 tout seul sur le rd point entouré par du 17-D202-10+400 TRONROUT0000000032864482
- affectation des points aux lignes : vérifier qu'au moins 1 lignes dans la distance eéquivalente au buffer présete bien le même numero de voie que le compteur
TRONROUT0000000032877685 avec 17-D107-62+282 

In [ ]:
%reset_selective -f df_linearisee_fin_glob
#on ne prend que les points de comptages présentant le mm nom de voie
gdf_variation_idcpt_ok=gdf_variation_idcpt.loc[gdf_variation_idcpt.apply(lambda x : 
                        x['id_cpt_lin'].split('-')[1] == x['id_cpt_pt'].split('-')[1],axis=1)].copy()

#isoler les lignes qui sont relaticves à l'id_cpt_lin et trouver tout les points concerné. si que 1 point de type permanent on passe
for id_cpt_a_test in gdf_variation_idcpt_ok.id_cpt_lin.unique() : 
    print(id_cpt_a_test)
#id_cpt_a_test='17-D939-88+0'
    lgn_idcpt_lin=gdf_traf.loc[gdf_traf['id_comptag']==id_cpt_a_test].copy()
    liste_new_pt=gdf_variation_idcpt_ok.loc[gdf_variation_idcpt_ok['id_cpt_lin']==id_cpt_a_test].id_cpt_pt.unique()
    if (gdf_pt_cpt.loc[gdf_pt_cpt.id_comptag.isin(liste_new_pt)].type_poste=='permanent').all() :
        print('que permanent')
        continue

    #trouver la ligne la plus proche relative à chaque nouveau point
    gdf_variation_idcpt_ok_geom=gdf_variation_idcpt_ok.merge(gdf_pt_cpt[['id_comptag', 'geom']], left_on='id_cpt_pt', right_on='id_comptag').rename(columns={
        'geom' : 'geom_id_cpt_pt'}) # pour trouver la ligne qui suppotrt ele pt j'ai besoin da le distance au pt de comptage
    gdf_variation_idcpt_ok_geom['dist_id_cpt']=gdf_variation_idcpt_ok_geom.apply(lambda x : x['geometry'].distance(x['geom_id_cpt_pt']), axis=1)

    t1=gdf_variation_idcpt_ok_geom.loc[gdf_variation_idcpt_ok_geom['id_cpt_pt'].isin(liste_new_pt)]
    df_lgn_proche_cpt=t1.loc[t1.groupby(['id_cpt_pt'])['dist_id_cpt'].transform(min)==t1['dist_id_cpt']]
    #si la distance minimale entre le nouveau point et le trocnon qui le supporte est sup à 0.5 (paramètre du buffer au dessus) ça veut dire que pb, dc on passe
    if (df_lgn_proche_cpt.dist_id_cpt>0.5).all() : 
        print('pb distance')
        continue

    #trouver les lignes qui intersectent celles de l'idcompatg etudie dont l'importance est = ou sup
        #list des source / target de l'idcomptage
    list_src_tgt=lgn_idcpt_lin.source.tolist()+lgn_idcpt_lin.target.tolist()
    df_lgn_interscts=gdf_traf.loc[((gdf_traf['source'].isin(list_src_tgt)) | (gdf_traf['target'].isin(list_src_tgt))) &
                                    (~gdf_traf.index.isin(lgn_idcpt_lin.index.tolist()))].copy()
    df_lgn_interscts_merge=gp.sjoin(df_lgn_interscts,lgn_idcpt_lin, op='intersects')
    df_lgn_interscts_filtre=df_lgn_interscts_merge.loc[df_lgn_interscts_merge.apply(lambda x : int(x['importance_left']) <= int(x['importance_right']), axis=1)].copy()
    nom_attribut=[a for a  in df_lgn_interscts_filtre.columns if a[-4:]=='left']
    df_lgn_interscts_fin=df_lgn_interscts_filtre.rename(columns={n : n[:-5] for n in nom_attribut})[[c for c in df_lgn_interscts.columns]]

    #faire un graph avec les lignes de l'id_cpt_lin et celles qui interectent avec la bonne importance
        #regrouper l'ensemble des lignes dans une df 
    ligne_pr_graph=gdf_traf.loc[gdf_traf['id_ign'].isin(df_lgn_interscts.id_ign.tolist()+lgn_idcpt_lin.id_ign.tolist())]
    ligne_pr_graph_transfert=ligne_pr_graph.copy()
    ligne_pr_graph_transfert['geom']=ligne_pr_graph_transfert.apply(lambda x : dumps(x.geometry), axis=1)

    #preparer le lineaire pour graph
    ligne_pr_graph=pd.concat([df_lgn_interscts_fin, lgn_idcpt_lin], sort=False, axis=0)
        #gestion des doucblons id_ign
    ligne_pr_graph.drop_duplicates('id_ign', inplace=True)
        #si doublons id restant on remplace l'id
    if ~ligne_pr_graph.loc[ligne_pr_graph.duplicated('id', keep=False)].empty : 
        ligne_pr_graph=ligne_pr_graph.reset_index().drop('index', axis=1).copy()
        ligne_pr_graph.id=ligne_pr_graph.index
    ligne_pr_graph_transfert=ligne_pr_graph.copy()
    ligne_pr_graph_transfert['geom']=ligne_pr_graph_transfert.apply(lambda x : dumps(x.geometry), axis=1)

    #faire un graph à partir de ces lignes
    bdd='gti_otv'
    schema='public'
    table='graph_temp'
    table_vertex='graph_temp_vertices_pgr'
    with ct.ConnexionBdd(bdd) as c:
        metadata = MetaData(schema=schema)
        #supprimer table si elle existe
        rqt=f"drop table if exists {schema}.{table} ; drop table if exists {schema}.{table_vertex} "
        c.sqlAlchemyConn.execute(rqt)

        #creer nouvelle table
        graph = Table('graph_temp', metadata,
            Column('id', Integer, primary_key=True),
            Column('id_ign', String),
            Column('importance', String),
            Column('numero', String),
            Column('nature', String),
            Column('codevoie_d', String),
            Column('sens', String),
            Column('source', Integer),
            Column('target', Integer),
            Column('long_km', Float),
            Column('geom',Geometry()))
        metadata.create_all(c.engine)
        ligne_pr_graph_transfert[['id','id_ign','importance','numero','nature','codevoie_d','source','sens','target','long_km','geom']].to_sql(
            table,c.sqlAlchemyConn,schema=schema,if_exists='append', index=False )
        #creer le graph
        rqt_creation_graph=f"""update {schema}.{table} set geom=st_Multi(st_setsrid(geom,2154)), source=null, target=null ; 
                             select pgr_createTopology('{schema}.{table}', 0.001,'geom')"""
        c.sqlAlchemyConn.execute(rqt_creation_graph)
        rqt_anlyse_graph=f"SELECT pgr_analyzeGraph('{schema}.{table}', 0.001,\'geom\')"
        c.curs.execute(rqt_anlyse_graph)#je le fait avec psycopg2 car avec sql acchemy ça ne passe pas
        c.connexionPsy.commit()

    #appel des fonction de carac des lignes 
    df=import_donnes_base('gti_otv','public', 'graph_temp','graph_temp_vertices_pgr')
    df2_chaussees=df.loc[df.nature.isin(['Autoroute', 'Quasi-autoroute', 'Route à 2 chaussées'])]
    df_avec_rd_pt,carac_rd_pt,lign_entrant_rdpt=identifier_rd_pt(df)
    df_lignes=df_avec_rd_pt.set_index('id_ign')#mettre l'id_ign en index

    #ensuite l'idée c'est de créer un dico avec pr chauqe point de comptage que l'on souhaite lineariser le recenesmeent de  : 
    #la ligne qui supporte le pt, le numero du tronc elem, les lignes qui compose le tronc elem
    #on faitd'abords les points ponctuels, puis le permanent pour pouvoir les ordonner avec un drapeau
    dico_ligne_pt_base=df_lgn_proche_cpt.drop_duplicates(['id_cpt_lin','id_cpt_pt' ])[['id_cpt_pt',
                            'id_ign','id_tronc_e']].set_index('id_cpt_pt').to_records()
    dico_ligne_pt={k[0] : {'ligne_base': k[1],'trc_elem':k[2],'lgn_trc_elem' : gdf_traf_pt.loc[(gdf_traf_pt['id_tronc_e']==k[2]) & 
                                                                            (gdf_traf_pt.id_ign.isin(lgn_idcpt_lin.id_ign.tolist()))].id_ign.tolist(), 'priorite':2} 
                   for k in dico_ligne_pt_base}
    dico_ligne_pt[id_cpt_a_test]={'ligne_base' : gdf_traf_pt.loc[(gdf_traf_pt['type_poste'].isin(['permanent','tournant'])) & 
                                                                (gdf_traf_pt['id_cpt_lin']==id_cpt_a_test)].id_ign.values[0],
                                 'trc_elem':gdf_traf_pt.loc[(gdf_traf_pt['type_poste'].isin(['permanent','tournant'])) & 
                                                                (gdf_traf_pt['id_cpt_lin']==id_cpt_a_test)].id_tronc_e.values[0], 
                                 'lgn_trc_elem' : gdf_traf_pt.loc[(gdf_traf_pt['id_tronc_e']==gdf_traf_pt.loc[(gdf_traf_pt['type_poste'].isin(
                                     ['permanent','tournant'])) & (gdf_traf_pt['id_cpt_lin']==id_cpt_a_test)].id_tronc_e.values[0]) & 
                   (gdf_traf_pt['id_ign'].isin(df_lignes.index.to_list()))].id_ign.tolist(),
                                 'priorite':1}
        #dico des lignes relatives a un autre comptage
    dico_lign_cpt_pct={k:[a for kd in dico_ligne_pt.keys() if kd!=k for a in dico_ligne_pt[kd]['lgn_trc_elem']] for k in dico_ligne_pt.keys()}

    #attention, la fonction regrouper_troncon ne marche pas si la ligne de base est un rd point.
    #il faut dc vérifier ce point et dans le cas où ça arrive prendre un ligne des lgn_trc_elem qui ne soit pas sur un rdpoint
    if not carac_rd_pt.empty : 
        list_id_ign_rd_pt=[a for l_rdpt in carac_rd_pt.id_ign_rdpt.tolist() for a in l_rdpt]
        for k in dico_ligne_pt.keys() : 
            if dico_ligne_pt[k]['ligne_base'] in list_id_ign_rd_pt : 
                dico_ligne_pt[k]['ligne_base']=[e for e in dico_ligne_pt[k]['lgn_trc_elem'] if e not in list_id_ign_rd_pt][0]

    #pour chaque ligne : on fait tourner la fonction de tronc elem à partir du graph avec les importance sup ou égale, 
    #d'abord pour le cpt permanent, ensuite pour les autres, avec prise necompte des lignes des tronc elem des autres compteurs. 
        #list des id_ign des tronc_elem
    liste_id_ign_te=[a  for k, v in dico_ligne_pt.items() for a in v['lgn_trc_elem']]
    lignes_traitees=[]
        #utilisret les fnctions de carac
    try : 
        for k in sorted(dico_ligne_pt.keys(), key=lambda x: (dico_ligne_pt[x]['priorite'])): 
            ligne_comp=[x for x in regrouper_troncon([dico_ligne_pt[k]['ligne_base']],df_avec_rd_pt,carac_rd_pt, df2_chaussees,
                                                       dico_lign_cpt_pct[k]+lignes_traitees)[0].id.tolist() 
                        if x in df_lignes.index.tolist()]
            #print(f"id_cpt : {k},'\n',lignes_traitees : {lignes_traitees},'\n', lignes de base : {ligne_comp}")
            lignes_traitees+=ligne_comp
            dico_ligne_pt[k]['lignes_comp_base']=[a for a in ligne_comp if a in lgn_idcpt_lin.id_ign.to_list()]
    except KeyError : 
        print(f'pb sur id_comptag : {id_cpt_a_test}')

    #a ce stade, si le cpt perm est séparé par une voie d'importance sup ou =, il y a discontinuité, dc il faut 
        #-relevé les discontinuite
    ids_affectes=[a for k in dico_ligne_pt.keys() for a in dico_ligne_pt[k]['lignes_comp_base']+dico_ligne_pt[k]['lgn_trc_elem']]
    df_non_affecte=lgn_idcpt_lin.loc[~lgn_idcpt_lin.id_ign.isin(ids_affectes)]

    #tant que des discont existent
    while not df_non_affecte.empty : 
        for k in sorted(dico_ligne_pt.keys(), key=lambda x: (dico_ligne_pt[x]['priorite'])):
            #discontinuité qui touche le cpt : 
            list_ids_cpt=list(set(dico_ligne_pt[k]['lignes_comp_base']+dico_ligne_pt[k]['lgn_trc_elem']))
            src_tgt_affecte=(lgn_idcpt_lin.loc[lgn_idcpt_lin.id_ign.isin(list_ids_cpt)].source.tolist() + 
                    lgn_idcpt_lin.loc[lgn_idcpt_lin.id_ign.isin(list_ids_cpt)].target.tolist() )
            ids_tch_cpt=df_non_affecte.loc[(df_non_affecte.source.isin(src_tgt_affecte)) | (df_non_affecte.target.isin(src_tgt_affecte))].id_ign.tolist()
            if not ids_tch_cpt :
                continue
            print (k, ids_tch_cpt)
            #pour chaque ligne on récupère tout les troncons elem et on ajoute au dico
            try : 
                ids_a_ajouter=[a for b in [regrouper_troncon([e],df_avec_rd_pt,carac_rd_pt, df2_chaussees,
                 ids_affectes)[0].id.tolist() for e in ids_tch_cpt] for a in b]
            except PasAffectationError :#dans le cas par exemple où tout les lignes qui restent sont des rd ppoint
                if (~df_non_affecte.id_rdpt.isna()).all() and len(df_non_affecte.id_rdpt.unique())==1 : 
                    ids_a_ajouter=df_non_affecte.id_ign.tolist()
            dico_ligne_pt[k]['lignes_comp_base']+=ids_a_ajouter
            ids_affectes=[a for k in dico_ligne_pt.keys() for a in dico_ligne_pt[k]['lignes_comp_base']+dico_ligne_pt[k]['lgn_trc_elem']]
            df_non_affecte=lgn_idcpt_lin.loc[~lgn_idcpt_lin.id_ign.isin(ids_affectes)]

    # mettre en forme et creation df de correspondance finale : 
    df_linearisee=pd.DataFrame(index=dico_ligne_pt.keys(), data=dico_ligne_pt.values())
    df_linearisee['lignes_comp_fin']=df_linearisee.apply(lambda x : x['lignes_comp_base'] + x['lgn_trc_elem'], axis=1 )
    dico_inverse=df_linearisee[['lignes_comp_fin']].to_dict()
    dico_inverse={a:k   for k, val  in dico_inverse['lignes_comp_fin'].items() for a in val}
    df_linearisee_fin=pd.DataFrame(index=dico_inverse.keys(), data=dico_inverse.values(), columns=['id_cpt_fin'])

    try : 
        df_linearisee_fin_glob=pd.concat([df_linearisee_fin_glob,df_linearisee_fin], axis=0, sort=False)
    except NameError : 
        df_linearisee_fin_glob=df_linearisee_fin.copy()
df_linearisee_fin_glob.to_csv(r'Q:\DAIT\TI\DREAL33\2018\C18SA00003_OTR-linearisation_NA_(JM)\Production\Travail\Donnees_produites\fichiers_temp_travail\test_lin_pt_existant.csv')

# pour les nouveaux points à linéariser 
**L'idée c'est d'isoler la voie concernée, en ne conservant que le troncons qui n'ont pas d'id_comptage. ensuite on fait le tour de tout les pt dispo, on crée un graph sur la base du numero de voie et des troncon qui intersectent et qui soit ont une importance sup ou =, soit sont compté. Puis on propage sur ce graph** <br> <br>
**ATTENTION DES ERREURS DE LINEARISATION EN 2016 :** <br>
- propagation non arreté à la 1èer intersection mais à la deuxième (à l'Est de l'id_comptag) : TRONROUT0000000032986160 linearisee avec 17-D129-57+510
- rd point linéarisé mais aucun autre troncon avec lui : 17-D202-9+950 tout seul sur le rd point entouré par du 17-D202-10+400 TRONROUT0000000032864482
- affectation des points aux lignes : vérifier qu'au moins 1 lignes dans la distance eéquivalente au buffer présete bien le même numero de voie que le compteur
TRONROUT0000000032877685 avec 17-D107-62+282 

In [159]:
#%reset_selective -f df_linearisee_fin_glob

#on ne garde que les points relatif à leur voie
gdf_creation_idcpt_ok=gdf_creation_idcpt.loc[gdf_creation_idcpt.apply(lambda x : x['numero']==x['id_cpt_pt'].split('-')[1],axis=1)].copy()

In [163]:
#on isole un point de comptage et on trouve tout les points relatifs à cette voie : 
pt_traites=[]
listAffectation=[]
for id_cpt_a_test in gdf_creation_idcpt_ok.id_cpt_pt.unique() :
    #drapeau si point deja traite dans le cadre d'un autre point sur la mm voie
    print(id_cpt_a_test)
    if id_cpt_a_test in pt_traites : 
        continue
    numero=id_cpt_a_test.split('-')[1]
    df_tt_cpt_route=gdf_creation_idcpt_ok.loc[gdf_creation_idcpt_ok['numero']==numero]
    liste_new_pt=df_tt_cpt_route.id_cpt_pt.unique()

    #on recherche la ligne la plus proche pour cahque pt de comptage
    gdf_creation_idcpt_ok_geom=df_tt_cpt_route.merge(gdf_pt_cpt[['id_comptag', 'geom']], 
                left_on='id_cpt_pt', right_on='id_comptag').rename(columns={'geom' : 'geom_id_cpt_pt'})
    gdf_creation_idcpt_ok_geom['dist_id_cpt']=gdf_creation_idcpt_ok_geom.apply(lambda x : x['geometry'].distance(x['geom_id_cpt_pt']), axis=1)

    t1=gdf_creation_idcpt_ok_geom.loc[gdf_creation_idcpt_ok_geom['id_cpt_pt'].isin(liste_new_pt)]
    df_lgn_proche_cpt=t1.loc[t1.groupby(['id_cpt_pt'])['dist_id_cpt'].transform(min)==t1['dist_id_cpt']]
    
    #toute les lignes du numero de voie
    df_troncon_numero=gdf_traf.loc[gdf_traf['numero']==numero]
    #sans les lignes qui ont déjà un id_comptag
    df_troncon_numero=df_troncon_numero.loc[df_troncon_numero['id_comptag'].isna()].copy()
    list_src_tgt_tronc_num=df_troncon_numero.source.tolist() + df_troncon_numero.target.tolist()
    
    #trouver les lignes d'importance sup ou = qui intersectent
    df_lgn_interscts=gdf_traf.loc[((gdf_traf['source'].isin(list_src_tgt_tronc_num)) | (gdf_traf['target'].isin(list_src_tgt_tronc_num))) &
                                    (~gdf_traf.index.isin(df_troncon_numero.index.tolist())) & (gdf_traf['importance']!='NC')].copy()
    df_lgn_interscts_merge=gp.sjoin(df_lgn_interscts,df_troncon_numero, op='intersects')
    df_lgn_interscts_filtre=df_lgn_interscts_merge.loc[df_lgn_interscts_merge.apply(lambda x : int(x['importance_left']) <= int(x['importance_right']), axis=1)].copy()
    nom_attribut=[a for a  in df_lgn_interscts_filtre.columns if a[-4:]=='left']
    df_lgn_interscts_fin=df_lgn_interscts_filtre.rename(columns={n : n[:-5] for n in nom_attribut})[[c for c in df_lgn_interscts.columns]]
    #trouver les lignes avec un id_comptag qu iintersectent
    df_lgn_id_cpt_lin=gdf_traf.loc[~gdf_traf.id_comptag.isna()].copy()
    df_lgn_id_cpt_lin=df_lgn_id_cpt_lin.loc[(df_lgn_id_cpt_lin['source'].isin(list_src_tgt_tronc_num)) | 
                                            (df_lgn_id_cpt_lin['target'].isin(list_src_tgt_tronc_num))].copy()
    
    #preparer le lineaire pour graph
    ligne_pr_graph=pd.concat([df_lgn_interscts_fin, df_lgn_id_cpt_lin,df_troncon_numero], sort=False, axis=0)
    ligne_pr_graph.drop_duplicates('id_ign', inplace=True)
        #si doublons id restant on remplace l'id
    if ~ligne_pr_graph.loc[ligne_pr_graph.duplicated('id', keep=False)].empty : 
        ligne_pr_graph.reset_index(drop=True,inplace=True)
        ligne_pr_graph.id=ligne_pr_graph.index
    ligne_pr_graph_transfert=ligne_pr_graph.rename(columns={'id_voie_d':'codevoie_d'}).copy()
    ligne_pr_graph_transfert['geom']=ligne_pr_graph_transfert.apply(lambda x : dumps(x.geometry), axis=1)

    creer_graph(ligne_pr_graph_transfert[['id','id_ign','importance','numero','nature','codevoie_d','source','sens','target','long_km','geometry']],
               'local_otv_boulot')
        
        #appel des fonction de carac des lignes 
    df=import_donnes_base('local_otv_boulot','public', 'graph_temp','graph_temp_vertices_pgr')
    df2_chaussees=df.loc[df.nature.isin(['Type autoroutier', 'Route à 2 chaussées'])]
    df_avec_rd_pt,carac_rd_pt,lign_entrant_rdpt=identifier_rd_pt(df)
    df_lignes=df_avec_rd_pt.set_index('id_ign')#mettre l'id_ign en index
    
    #creation d'un dico des ptde base
    dico_ligne_pt_base=df_lgn_proche_cpt.drop_duplicates(['id_cpt_lin','id_cpt_pt' ])[['id_cpt_pt',
                            'id_ign','id_tronc_e']].set_index('id_cpt_pt').to_records()
    dico_ligne_pt={k[0] : {'ligne_base': k[1],'trc_elem':k[2],'lgn_trc_elem' : gdf_traf_pt.loc[(gdf_traf_pt['id_tronc_e']==k[2]) & 
                                                                                (gdf_traf_pt.id_ign.isin(df_troncon_numero.id_ign.tolist()))].id_ign.tolist()
                           , 'priorite':2} 
                       for k in dico_ligne_pt_base}
    
    #dico des lignes relatives a un autre comptage
    dico_lign_cpt_pct={k:[a for kd in dico_ligne_pt.keys() if kd!=k for a in dico_ligne_pt[kd]['lgn_trc_elem']] for k in dico_ligne_pt.keys()}
    
    #attention, la fonction regrouper_troncon ne marche pas si la ligne de base est un rd point.
    #il faut dc vérifier ce point et dans le cas où ça arrive prendre un ligne des lgn_trc_elem qui ne soit pas sur un rdpoint
    if not carac_rd_pt.empty : 
        list_id_ign_rd_pt=[a for l_rdpt in carac_rd_pt.id_ign_rdpt.tolist() for a in l_rdpt]
        for k in dico_ligne_pt.keys() : 
            if dico_ligne_pt[k]['ligne_base'] in list_id_ign_rd_pt : 
                dico_ligne_pt[k]['ligne_base']=[e for e in dico_ligne_pt[k]['lgn_trc_elem'] if e not in list_id_ign_rd_pt][0]
                
    #pour chaque ligne : on fait tourner la fonction de tronc elem à partir du graph avec les importance sup ou égale, 
    #d'abord pour le cpt permanent, ensuite pour les autres, avec prise necompte des lignes des tronc elem des autres compteurs. 
        #list des id_ign des tronc_elem
    liste_id_ign_te=[a  for k, v in dico_ligne_pt.items() for a in v['lgn_trc_elem']]
    lignes_traitees=[]
        #utilisret les fnctions de carac
    try : 
        for k in sorted(dico_ligne_pt.keys(), key=lambda x: (dico_ligne_pt[x]['priorite'])): 
            ligne_comp=[x for x in regrouper_troncon([dico_ligne_pt[k]['ligne_base']],df_avec_rd_pt,carac_rd_pt, df2_chaussees,
                                                       dico_lign_cpt_pct[k]+lignes_traitees)[0].id.tolist() 
                        if x in df_lignes.index.tolist()]
            #print(f"id_cpt : {k},'\n',lignes_traitees : {lignes_traitees},'\n', lignes de base : {ligne_comp}")
            lignes_traitees+=ligne_comp
            dico_ligne_pt[k]['lignes_comp_base']=[a for a in ligne_comp if a in df_troncon_numero.id_ign.to_list()]
    except KeyError : 
        print(f'pb sur id_comptag : {id_cpt_a_test}')
        
    # mettre en forme et creation df de correspondance finale : 
    df_linearisee=pd.DataFrame(index=dico_ligne_pt.keys(), data=dico_ligne_pt.values())
    df_linearisee['lignes_comp_fin']=df_linearisee.apply(lambda x : x['lignes_comp_base'] + x['lgn_trc_elem'], axis=1 )
    dico_inverse=df_linearisee[['lignes_comp_fin']].to_dict()
    dico_inverse={a:k   for k, val  in dico_inverse['lignes_comp_fin'].items() for a in val}
    df_linearisee_fin=pd.DataFrame(index=dico_inverse.keys(), data=dico_inverse.values(), columns=['id_cpt_fin'])
    listAffectation.append(df_linearisee_fin)     
    pt_traites+=list(dico_ligne_pt.keys())
df_linearisee_fin_glob=pd.concat(listAffectation)

17-D141E4-1+385


c:\users\martin.schoreisz\appdata\local\programs\python\python38\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


creer_graph : donnees mise en forme, connexion ouverte ; 2020-12-10 12:11:00.964791
creer_graph : donnees transferees dans la base postgis ; 2020-12-10 12:11:01.026657
creer_graph : geometrie modifiee ; 2020-12-10 12:11:01.066093
creer_graph : topologie cree ; 2020-12-10 12:11:01.348914
creer_graph : graph cree ; 2020-12-10 12:11:01.566327
0 2020-12-10 12:11:03.255134 nb lignes traitees : 0
17-D215E2-1+650


c:\users\martin.schoreisz\appdata\local\programs\python\python38\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


creer_graph : donnees mise en forme, connexion ouverte ; 2020-12-10 12:11:04.242980
creer_graph : donnees transferees dans la base postgis ; 2020-12-10 12:11:04.318742
creer_graph : geometrie modifiee ; 2020-12-10 12:11:04.405152
creer_graph : topologie cree ; 2020-12-10 12:11:04.553785
creer_graph : graph cree ; 2020-12-10 12:11:04.713381
0 2020-12-10 12:11:06.209817 nb lignes traitees : 0
17-D106-3+20


...

creer_graph : donnees mise en forme, connexion ouverte ; 2020-12-10 12:16:26.177310
creer_graph : donnees transferees dans la base postgis ; 2020-12-10 12:16:26.290008
creer_graph : geometrie modifiee ; 2020-12-10 12:16:26.351816
creer_graph : topologie cree ; 2020-12-10 12:16:27.287687
creer_graph : graph cree ; 2020-12-10 12:16:27.508103
0 2020-12-10 12:16:29.094841 nb lignes traitees : 0
0 2020-12-10 12:16:29.149695 nb lignes traitees : 0
0 2020-12-10 12:16:29.395095 nb lignes traitees : 0
0 2020-12-10 12:16:29.827893 nb lignes traitees : 0
17-D110-2+980
17-D136-33+600
17-D118-37+680
17-D142-0+900
17-D145-64+150
17-D149-1+740
17-D253-3+300
17-D910-1+936
17-D128-39+970
17-D136-9+570
17-D118-58+300
17-D236E1-2+250


c:\users\martin.schoreisz\appdata\local\programs\python\python38\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


creer_graph : donnees mise en forme, connexion ouverte ; 2020-12-10 12:16:30.860205
creer_graph : donnees transferees dans la base postgis ; 2020-12-10 12:16:30.942607
creer_graph : geometrie modifiee ; 2020-12-10 12:16:31.001881
creer_graph : topologie cree ; 2020-12-10 12:16:31.134503
creer_graph : graph cree ; 2020-12-10 12:16:31.372920
0 2020-12-10 12:16:32.617577 nb lignes traitees : 0
17-D145-111+750
17-D117E1-11+490
17-D119-27+200
17-D145-111+311
17-D110-1+970
17-D136-10+120
17-D119-27+540
17-D117E1-11+320
17-D149-7+140
17-D140E3-0+30


c:\users\martin.schoreisz\appdata\local\programs\python\python38\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


creer_graph : donnees mise en forme, connexion ouverte ; 2020-12-10 12:16:33.476511
creer_graph : donnees transferees dans la base postgis ; 2020-12-10 12:16:33.527573
creer_graph : geometrie modifiee ; 2020-12-10 12:16:33.563339
creer_graph : topologie cree ; 2020-12-10 12:16:33.737467
creer_graph : graph cree ; 2020-12-10 12:16:34.012686
0 2020-12-10 12:16:35.148888 nb lignes traitees : 0
17-D139-7+50
17-D127-47+840


In [ ]:
gdf_lin.merge(df_linearisee_fin_glob.rename_axis('id_ign').reset_index(), on='id_ign').to_file(
    r'C:\Users\martin.schoreisz\Documents\AffairesEnCours\OTV\17\temp_linearisation\affectationV1.shp')

In [121]:
'17-D129E2-2+635' in gdf_creation_idcpt_ok.id_cpt_pt.unique()

True